In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


Load dataset

In [2]:
def loadMatFile(file_path, file_name, key):
    """
    Load a mat file and return an item of the dictionary loaded.
    """    
    # read mat file dictionary
    dictionary = scipy.io.loadmat(file_path + file_name)
    
    # access item of a dictionary
    array = dictionary[key]
    
    return array

In [12]:
# Load hologram dataset
path = 'C:\\Users\\flucasamar\\Desktop\\Github\\computerGeneratedHolography\\'
file_path = path + '\\output\\dataset\\'

file_name = 'hDataset.mat'
key = 'hDataset'

# Load dictionary
dat = loadMatFile(file_path, file_name, key)
print(dat.shape)

# Number of holograms
nb_holograms = dat.shape[2]

# Number of class
nb_class = 5

# Number of holograms per class
nb_holograms_class = int(nb_holograms/nb_class)

print('Total number of holograms: ' + str(nb_holograms))
print('Number of holograms per class: ' + str(nb_holograms_class))

(200, 200, 500)
Total number of holograms: 500
Number of holograms per class: 100


Pre-processing

In [28]:
# The 1 signify that the images are greyscale
X_train = dat.reshape(200, 200, 500, 1)

# Compute array of targets
Y_array = np.ones([nb_holograms,])

pos = 0
for c in range(nb_class):
    for h in range(nb_holograms_class):
        Y_array[pos] = c
        pos = pos + 1
print(Y_array.shape)

# One-hot encode target column
Y_train = to_categorical(Y_array)
print(Y_train.shape)

(500,)
(500, 5)


Building the model

In [29]:
# Create model
model = Sequential() # allows build a model layer by layer

# Add model layers

# Conv2D layer: 
# 64 nodes, 3x3 filter matrix, Rectified Linear Activation as activation function,
# shape of each input (200, 200, 1,) with 1 signifying images are greyscale
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(200,200,1))) 

# 32 nodes
model.add(Conv2D(32, kernel_size=3, activation='relu'))

# Flatten layer: connection between the convolution and dense layers
model.add(Flatten())

# Dense layer: used for the output layer
# 5 nodes for the output layer, one for each possible outcome (1-5)
# 'softmax' as activation function, it makes the output sump up to 1 so the output
# can be interpreted as probalities
model.add(Dense(5, activation='softmax'))

In [30]:
# Three parameters:
# optmizer: 'adam'
# loss function: 'categorical_crossentropy', the most common choice for classification
# metrics: 'accuracy', to see the accuracy score
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
%%time

# Number of epochs: number of tmes the model wil cycle trough the data
model.fit(X_train, Y_train, validation_data=(X_train, Y_train), epochs=30)

ValueError: Error when checking input: expected conv2d_5_input to have shape (200, 200, 1) but got array with shape (200, 500, 1)